In [1]:
import numpy as np
import pandas as pd

In [2]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec.load('wiki_w2v.model')

In [3]:
def sentence_vec(sentence,model):  #待判别语句，词向量模型
    import jieba
    senvec = np.zeros((1,300))
    newsen = jieba.cut(sentence)
    length = 0
    for i in newsen:
        try:
            senvec += model[i].reshape((1,300))
            length += 1
        except KeyError:
            continue 
    return senvec/length  #平均词向量

In [4]:
def sentence_decision(sentence,model=model,clf='erji lbfgs_4000_(100, 50)network.pickle'):  #待判别语句，分类模型，词向量模型
    import pickle
    import warnings
    warnings.filterwarnings('ignore')
    re = sentence_vec(sentence,model)
    if np.isnan(re)[0][0]:  #注意nan的处理,识别不出按负类处理
        return np.array([0.])   
    with open(clf, 'rb') as fr:
        if 'svm' in clf:  #如果要用svc的话，需要对输入进行降维
            with open('erji_pca_20.pickle','rb') as pr: 
                pcapatten = pickle.load(pr)
                re = pcapatten.transform(re)
        cl = pickle.load(fr)      
    return cl.predict(re)   #语句类别

In [5]:
dic_li=[('外观',0),('质感',0),('灵敏度',0),('舒适度',0),('续航',0),('音质',0),('特色',0),('正版',0),('保价',0),('包装',0),('客服',0),('售后',0)]
dic_p=dict(dic_li)
dic_n=dict(dic_li)
demo=['外观','质感','灵敏度','舒适度','续航','音质','特色','正版','保价','包装','客服','售后']

In [6]:
def demo_sim(sen,dic_p,dic_n,demo=demo,model=model):
    import jieba
    import re
    dic_pos={}
    dic_neg={}
    #若想批量处理的话，sen传一个大句子，遍历
    senlist=sen.split("\n")
    for d in demo:
        dic_pos[d]=dic_p[d]
        dic_neg[d]=dic_n[d]
        for s in senlist:  #s含有特定评价指标的短句子
            words=list(jieba.cut(s,cut_all=True))
            for w in words:
                w= re.sub("[A-Za-z0-9\!\：\？\?\!\！\.\。\，\、\&\n]", "", w)
                if w!="":
                    try:
                        if model.similarity(w,d)>= 0.7:#指标的匹配
                            if int(sentence_decision(s)[0])==1:  #包含指标的短句情感判断
                                dic_pos[d]+=1
                                print('pos '+w)
                            else:
                                dic_neg[d]+=1
                                print('neg '+w)
                    except KeyError:
                        print('no '+w)
                        continue 
    return dic_pos,dic_neg

In [7]:
a,b=demo_sim("""这款耳机的做工非常的好，然后正版的音质也非常好，续航能力特别强，灵敏度非常高，弹窗都很方便，舒适度也非常好，我就喜欢坐地铁的时候，或者是听音乐的时候，自己放松的时候使用耳机，非常的不错，然后贴合性也非常好，挺舒适的，推荐大家购买""",
dic_p,dic_n
)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 1.458 seconds.
Prefix dict has been built successfully.
f:\python\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


FileNotFoundError: [Errno 2] No such file or directory: 'erji lbfgs_4000_(100, 50)network.pickle'

In [ ]:
a

In [ ]:
b

In [ ]:
c,d=demo_sim("""做工质感：一般般
音质音效：不错
舒适度：耳塞戴久了耳朵不舒服
续航能力：说好了可以听歌持续8小时，我听了三个小时都没有就没电了，**""",
a,b
)

In [ ]:
c,d